In [1]:
from google.colab import drive
# Mount Drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install flask-ngrok
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 15.3MB/s 
     |████████████████████████████████| 1.1MB 28.5MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=87613105de51baf1d8bb5e1ef6656b885c6613e6b062fc8d5c5bc77ff6250d79
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [13]:
%cd "/content/drive/My Drive/NLP Project/Deployment-flask"

[Errno 2] No such file or directory: 'file_path'
/content/drive/My Drive/NLP Project/Deployment-flask


To view the HTML page, click on the second link in the output console.

In [12]:
# Code referenced from https://github.com/krishnaik06/Deployment-flask

import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template
import pickle

import transformers
from transformers import DistilBertModel, DistilBertTokenizer

# basic pretrained model (case-insensitive)
MODEL_NAME = 'distilbert-base-uncased'
# Load pre-trained model
bert_model = DistilBertModel.from_pretrained(MODEL_NAME)
# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

app = Flask(__name__)
run_with_ngrok(app)
svc_model = pickle.load(open('model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    input_review = request.form.values()
    tokens = tokenizer.batch_encode_plus(input_review , 
                                         pad_to_max_length=True, max_length=200, truncation=True, 
                                         return_tensors="pt")
            
    outputs = bert_model(**tokens)
    final_features = outputs[0][:,0].detach().numpy()
    prediction = svc_model.predict(final_features)
    prob_list = svc_model.predict_proba(final_features)

    if prediction == ['positive']:
      result = "Positive"
      prob = round(prob_list[0,1]*100, 3)
    else:
      result = "Negative" 
      prob = round(prob_list[0,0]*100, 3)

    return render_template('index.html', prediction_text=f'The predicted sentiment of the review is {result} with {prob}% probability.')

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://557929ff7dac.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Aug/2020 15:17:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2020 15:17:23] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2020 15:17:24] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Aug/2020 15:17:29] "POST /predict HTTP/1.1" 200 -
